In [1]:
#Imports EdgarGarcia Coursera w3-1
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [2]:
#Wikipedia link get
wikipedia_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:64.0) Gecko/20100101 Firefox/64.0'}
wikipedia_page = requests.get(wikipedia_link, headers = headers)
wikipedia_page

<Response [200]>

## After get the link, extracts the data

In [3]:
#BeautifulSoup for scrapping easy
soup = BeautifulSoup(wikipedia_page.content, 'html.parser')
# tbody extract
table = soup.find('table').tbody
# tr in the table rows 
rows = table.find_all('tr')
#Columns
columns = [i.text.replace('\n', '')
           for i in rows[0].find_all('b')]
columns2 = [i.text.replace('\n', '')
           for i in rows[0].find_all('span')]

#columns to pd dataframe
df = pd.DataFrame(columns = columns)
df2 = pd.DataFrame(columns = columns2)


## All the rows most be extracted

## Create pd dataframe "df"

In [4]:
for i in range(0, len(rows)):
    tds = rows[i].find_all('b')
    if len(tds) == 9:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        df = df.append(pd.Series(values, index = columns), ignore_index = True)
        
# Borought loop
for i in range(0, len(rows)):
    tds1 = rows[i].find_all('span')
    if len(tds) == 9:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds1]
        df2 = df2.append(pd.Series(values, index = columns2), ignore_index = True)

## Toronto dataframe stored

In [ ]:
a = np.array(df)
a = a.reshape((-1, 1))
b = np.array(df2)
b = b.reshape((-1, 1))
code = pd.DataFrame({'Postcode': a[:, 0]})
borought = pd.DataFrame({'Borought': b[:, 0]})

dataset= pd.concat([code, borought.reindex(code.index)], axis=1)
#drop allraw Not assigned
indexNames = dataset[ dataset['Borought'] == 'Not assigned' ].index
dataset.drop(indexNames , inplace=True)

In [687]:
dataset

,Postcode,Borought
2,M3A,North York(Parkwoods)
3,M4A,North York(Victoria Village)
4,M5A,Downtown Toronto(Regent Park / Harbourfront)
5,M6A,North York(Lawrence Manor / Lawrence Heights)
6,M7A,Queen's Park / Ontario Provincial Government
...,...,...
160,M8X,Etobicoke(The Kingsway / Montgomery Road / Old...
165,M4Y,Downtown Toronto(Church and Wellesley)
168,M7Y,East TorontoBusiness reply mail Processing Cen...
169,M8Y,Etobicoke(Old Mill South / King's Mill Park / ...


In [36]:
dataset.to_csv('TorontoPostcodes.csv', index = False)

## Question 2 - Geocoder

In [6]:
!pip -q install geocoder
import geocoder # import geocoder

import time
start = time.time()

#Geocoder starts here
def get_latlng(arcgis_geocoder):
    
    # Start location on "None"
    lat_lng_coords = None
    
    #Loop for located    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords
#Geocoder ends here

end = time.time()
print("Time of execution: ", end - start)

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Time of execution:  0.00022268295288085938


In [7]:
sample = get_latlng('M4G')
sample

[43.70941386000004, -79.36309957799995]

In [8]:
gg = geocoder.geocodefarm(sample,method = 'reverse')
gg

<[FAILED, NO_RESULTS] Geocodefarm - Reverse [empty]>

In [17]:
import time
start = time.time()

postal_codes = dataset['Postcode']    
coordinates = [get_latlng(postal_code) for postal_code in postal_codes.tolist()]

end = time.time()
print("Time of execution: ", end - start, "seconds")

Time of execution:  0.0010352134704589844 seconds


## Toronto dataframe with coordinates stored

In [27]:
# Toronto dataframe with coordinates stored
df_toronto_loc = dataset

#latitude and longitude in the dataframe
df_toronto_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df_toronto_loc['Latitude'] = df_toronto_coordinates['Latitude']
df_toronto_loc['Longitude'] = df_toronto_coordinates['Longitude']

In [28]:
df_toronto_loc2= df_toronto_loc.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
(df_toronto_loc2['Latitude'] == 'NaN').sum()

0

In [29]:
df_toronto_loc2

,Postcode,Borought,Latitude,Longitude


In [30]:
df_toronto_loc2.to_csv('TorontoPostcodes.csv', index = False)

In [22]:
!pip -q install geopy
#conda install -c conda-forge geopy --yes 
print('geopy installed...')
#address into latitude and longitude 
from geopy.geocoders import Nominatim
print('Nominatim imported...')

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
geopy installed...
Nominatim imported...


In [23]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [24]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip -q install folium
print('folium installed...')
import folium # map rendering library
print('folium imported...')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
folium installed...
folium imported...


# Toronto Map

In [25]:
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 12)
map_toronto

# Here Visualize The Neighborhoods 

### Adding points markers to the map

In [26]:

for lat, lng, Borought in zip(df_toronto_loc2['Latitude'], 
                                            df_toronto_loc2['Longitude'],
                                            df_toronto_loc2['Borought']):
    label = '{}'.format(Borought)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
display(map_toronto)

## EdgarGarcia Coursera CApstine w3-1-2-3